In [1]:
from elftools.elf.elffile import ELFFile
from elftools.elf.sections import SymbolTableSection

In [2]:
fname = './elffuncs.o'
f = open(fname, 'rb')
elffile = ELFFile(f)

In [3]:
for sec in elffile.iter_sections():
    print(type(sec), sec.name)

<class 'elftools.elf.sections.NullSection'> 
<class 'elftools.elf.sections.Section'> .group
<class 'elftools.elf.sections.Section'> .group
<class 'elftools.elf.sections.Section'> .group
<class 'elftools.elf.sections.Section'> .group
<class 'elftools.elf.sections.Section'> .group
<class 'elftools.elf.sections.Section'> .text
<class 'elftools.elf.relocation.RelocationSection'> .rela.text
<class 'elftools.elf.sections.Section'> .data
<class 'elftools.elf.sections.Section'> .bss
<class 'elftools.elf.sections.Section'> .rodata.str1.8
<class 'elftools.elf.sections.Section'> .text._ZN3ElfD2Ev
<class 'elftools.elf.relocation.RelocationSection'> .rela.text._ZN3ElfD2Ev
<class 'elftools.elf.sections.Section'> .gcc_except_table._ZN3ElfD2Ev
<class 'elftools.elf.sections.Section'> .rodata.str1.1
<class 'elftools.elf.sections.Section'> .text.unlikely
<class 'elftools.elf.relocation.RelocationSection'> .rela.text.unlikely
<class 'elftools.elf.sections.Section'> .text.startup
<class 'elftools.elf.reloc

In [4]:
symtab = elffile.get_section_by_name('.symtab')
if symtab is None:
    print('Warning: The symbol table does not exist')

In [5]:
def is_nm_symbol(sym):
    return sym.name != '' and sym['st_info']['type'] != 'STT_FILE'

def nm_symbols(symtab):
    return [sym for sym in symtab.iter_symbols()
            if is_nm_symbol(sym)]

def is_nm_extern(sym):
    return sym['st_info']['bind'] != 'STB_LOCAL'

def is_nm_weak(sym):
    return sym['st_info']['bind'] == 'STB_WEAK'

def is_nm_strong(sym):
    return sym['st_info']['bind'] == 'STB_GLOBAL'

def is_nm_defined(sym):
    return sym['st_shndx'] != 'SHN_UNDEF'

def is_nm_func(sym):
    return sym['st_info']['type'] == 'STT_FUNC'

In [6]:
symtab['sh_type']

'SHT_SYMTAB'

In [7]:
symtab.num_symbols()

110

In [8]:
syms = nm_symbols(symtab)
len(syms)

78

In [9]:
from pprint import pprint
pprint([sym.name for sym in syms
                 if is_nm_extern(sym) and
                    is_nm_defined(sym) and
                    is_nm_func(sym) and
                    is_nm_strong(sym)])
for sym in syms:
    if is_nm_extern(sym) and is_nm_defined(sym) and is_nm_func(sym):
        d = dict(sym.entry)
        d['st_info'] = dict(d['st_info'])
        d['st_other'] = dict(d['st_other'])
        d['name'] = sym.name
        pprint(d)

['_ZN3Elf10printfuncsEv', 'main']
{'name': '_ZN3Elf10printfuncsEv',
 'st_info': {'bind': 'STB_GLOBAL', 'type': 'STT_FUNC'},
 'st_name': 444,
 'st_other': {'visibility': 'STV_DEFAULT'},
 'st_shndx': 6,
 'st_size': 1,
 'st_value': 224}
{'name': '_ZN3ElfD2Ev',
 'st_info': {'bind': 'STB_WEAK', 'type': 'STT_FUNC'},
 'st_name': 466,
 'st_other': {'visibility': 'STV_DEFAULT'},
 'st_shndx': 11,
 'st_size': 50,
 'st_value': 0}
{'name': '_ZN3ElfD1Ev',
 'st_info': {'bind': 'STB_WEAK', 'type': 'STT_FUNC'},
 'st_name': 520,
 'st_other': {'visibility': 'STV_DEFAULT'},
 'st_shndx': 11,
 'st_size': 50,
 'st_value': 0}
{'name': 'main',
 'st_info': {'bind': 'STB_GLOBAL', 'type': 'STT_FUNC'},
 'st_name': 532,
 'st_other': {'visibility': 'STV_DEFAULT'},
 'st_shndx': 17,
 'st_size': 1870,
 'st_value': 0}


In [10]:
#for sym in syms:
#    print(sym.entry, sym.name)
#len([sym.name for sym in syms if sym['st_info']['bind'] != 'STB_LOCAL'])
len([sym.name for sym in syms if sym['st_shndx'] != 'SHN_UNDEF'])

40